In [1]:
# 로그 + 타빌리
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Ollama 불러오기
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

model_name = 'llama3.2:1b'
ollama_url = 'http://localhost:11434'
llm = ChatOllama(model= model_name, base_url= ollama_url)
llm

ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')

In [34]:
### 사용자 제공 툴 (함수나 API기능을 LLM에게 제공)
# Tool Creation
from langchain_core.tools import tool

# LLM이 사용할 수 있게 함수를 제작한다 (파이썬 함수를 LLM 도구로 만든다)
@tool
def add(a:int, b:int)->int:
    """
    Add Two integer numbers together

    Args:
    a: First Integer
    b: Second Interger
    """
    return int(a) + int(b)

@tool
def multiply(a:int, b:int)->int:
    """
    Multiply Two integer numbers together

    Args:
    a: First Integer
    b: Second Interger
    """
    return int(a) * int(b)

In [35]:
# 더이상 일반 함수가 아니다
add.invoke({'a':1, 'b':32})

33

In [5]:
tools = [add, multiply]

tools_llm = llm.bind_tools(tools)
tools_llm

RunnableBinding(bound=ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': 'Add Two integer numbers together\n\nArgs:\na: First Integer\nb: Second Interger', 'parameters': {'properties': {'a': {}, 'b': {}}, 'required': ['a', 'b'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'multiply', 'description': 'Multiply Two integer numbers together\n\nArgs:\na: First Integer\nb: Second Interger', 'parameters': {'properties': {'a': {}, 'b': {}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [8]:
question = "what is 1 plus 2?"
tools_llm.invoke(question)

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-02-16T20:26:18.5769215Z', 'done': True, 'done_reason': 'stop', 'total_duration': 421786400, 'load_duration': 20177300, 'prompt_eval_count': 242, 'prompt_eval_duration': 6000000, 'eval_count': 22, 'eval_duration': 394000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-38b70381-29f2-427b-a684-d1ffe23c283d-0', tool_calls=[{'name': 'add', 'args': {'a': 1, 'b': 2}, 'id': 'cd920a53-6f57-466d-be87-20e0a40392de', 'type': 'tool_call'}], usage_metadata={'input_tokens': 242, 'output_tokens': 22, 'total_tokens': 264})

In [6]:
question = "what is 1 plus 2?"
tools_llm.invoke(question).tool_calls

[{'name': 'add',
  'args': {'a': 1, 'b': 2},
  'id': 'd6c72ef1-55fa-4520-bf60-7845ae219110',
  'type': 'tool_call'}]

In [7]:
question = "what is 1 muliplied by 2?"
tools_llm.invoke(question).tool_calls

[{'name': 'multiply',
  'args': {'a': 1, 'b': 2},
  'id': '05718870-96c3-461a-b847-92b9ce8ad67b',
  'type': 'tool_call'}]

In [26]:
### Calling In-Built Tool
# 검색 추가 (https://python.langchain.com/docs/integrations/tools/)
# pip install -U tavily-python

# Tavily검색 (API키 필요, 매달 1000개 무료)
from langchain_community.tools import TavilySearchResults

search = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
)

# 사용 (# 검색 결과를 LLM에게 제공할 수 있다)
question = "what is today's stock market news?"
response = search.invoke(question)
print(response)

[{'url': 'https://www.investors.com/news/stock-market-today-stock-market-news/', 'content': "Stock Market News And Analysis. The analysis you'll find in the Stock Market Today is based on over 130 years of market history and a detailed study of every top-performing stock since the 1880s."}, {'url': 'https://apnews.com/article/stock-markets-tariffs-trump-trade-66c4a754d716821ec8f65a15af9ad23c', 'content': 'Stock market today: Wall Street falls after worse inflation data NEW YORK (AP) — U.S. stocks are falling Wednesday after a report said inflation is unexpectedly getting worse for Americans. The inflation report suggests not only that pressure on U.S. households’ budgets is amplifying but also that traders on Wall Street were correct to forecast the Federal Reserve will deliver less relief for Americans through lower interest rates this year. AP AUDIO: Stock market today: Wall Street falls after worse inflation data The Fed had cut its main interest rate sharply from September through 

In [14]:
# 검색 결과를 LLM에게 제공할 수 있다
question = "2025년 2월 17일의 서울 날씨는?"
search.invoke(question)

[{'url': 'https://www.chosun.com/national/transport-environment/2025/02/17/EIM76NSDVJB63B5VVIYZEVKSVY/',
  'content': '오늘의 날씨 2025년 2월 17일 기온 크게 떨어지고 바람 불어 추워'},
 {'url': 'https://www.msn.com/ko-kr/weather/기상학/오늘의-날씨-오전-2025년-02월-17일/ar-AA1zajbF',
  'content': '2월 17일 월요일, 아침 날씨입니다.전국 기온이 영하권에 놓인 가운데 추운 날씨를 보이고 있습니다. 낮엔 기온이 오르면서 활동하기가 훨씬'},
 {'url': 'https://ko.climate-data.org/아시아/대한민국/서울/서울-718563/t/2월-2/',
  'content': '대륙 선택 국가 선택 지역 선택 장소 선택 서울 Climate-Data.org > 대한민국 > 서울 > 서울 > 2월 서울 2월의 날씨 개요: 서울 2월의 날씨 및 기후 서울 날씨 2월 // 날씨 평균 2월의 서울 날씨 온도 2월   -0.8°C 서울 Data: 1991 - 2021 최소 온도 (°C), 최고 온도 (°C), 강수량 (mm), 습도, 비오는 날. 향후 몇 달 동안의 서울 날씨 및 기후 서울 날씨 2월 // 날씨 평균 (°C)  강수량 / 강수량 (mm)  평균 일요일 시간 (시간) 1. 2월   -4 °C   1 °C    -8 °C   1.8 mm  0.0 2. 2월   -4 °C   1 °C    -8 °C   0.6 mm  0.0 3. 2월   -3 °C   1 °C    -7 °C   1.1 mm  0.0 4. 2월   -3 °C   2 °C    -7 °C   0.5 mm  0.0 기후 서울'},
 {'url': 'https://mossa.tistory.com/205',
  'content': '블로그 내 검색 검색 2025년 2월 날씨 전망: 2월 날씨 전국, 2월 날씨 지역 예보 확

In [17]:
### 위키피디아 검색
# pip install wikipedia
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# 검색 결과를 LLM에게 제공할 수 있다
question = "What is the capital of Republic of Korea?"
response = wikipedia.invoke(question)
print(response)

Page: South Korea
Summary: South Korea, officially the Republic of Korea (ROK), is a country in East Asia. It constitutes the southern half of the Korean Peninsula and borders North Korea along the Korean Demilitarized Zone, with the Yellow Sea to the west and the Sea of Japan to the east. Like North Korea, South Korea claims to be the sole legitimate government of the entire peninsula and adjacent islands. It has a population of 51.71 million, of which half live in the Seoul Metropolitan Area, the ninth most populous metropolitan area in the world; other major cities include Busan, Daegu, and Incheon.
The Korean Peninsula was inhabited as early as the Lower Paleolithic period. Its first kingdom was noted in Chinese records in the early seventh century BCE. From the mid first century BCE, various polities consolidated into the rival kingdoms of Goguryeo, Baekje, and Silla, with the lattermost unifying the peninsula for the first time in the late seventh century CE. The Goryeo dynasty (

In [25]:
# PubMed (의학 및 생명공학 관련 지식 검색)
# pip install xmltodict

from langchain_community.tools.pubmed.tool import PubmedQueryRun

search = PubmedQueryRun()

# 검색 결과를 LLM에게 제공할 수 있다
response = search.invoke("COVID-19")
print(response)

Published: 2025-02-16
Title: 60-day mortality and the role of SARS-CoV-2 in hospital admissions of immunocompromised patients during later Omicron period: a population-based study in Sweden.
Copyright Information: 
Summary::
BACKGROUND: Real-world data on hospitalised SARS-CoV-2-positive patients are important for post-pandemic preventive measures.
OBJECTIVES: This study aims to explore 60-day mortality of immunocompromised patients hospitalised in later Omicron period, accounting for the relevance of COVID-19 for hospital care.
METHODS: A retrospective population-based cohort study in Östergötland County, Sweden, included all adult patients with a positive SARS-CoV-2 PCR test within 3 weeks of hospital admission. Clinical data including functional level (combined assessment of frailty and performance status), and COVID-19's impact on hospital care were collected from medical records. An adjusted binary logistic regression model was applied for the main outcome of 60-day COVID-19-relat

In [27]:
### LLM으로 도구 호출하는 방법

# 위키피디아 검색를 LLM에 제공
@tool
def wikipedia_search(query):
    """
    Search wikipedia for general information

    Args:
    query: The search query
    """

    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    response = wikipedia.invoke(query)
    return response

# pubmed 검색
@tool
def pubmed_search(query):
    """
    Search pubmed for medical and life sciences queries.

    Args:
    query: The search query
    """

    search = PubmedQueryRun()
    response = search.invoke(query)
    return response

@tool
def tavily_search(query):
    """
    Search the web for realtime and latest information.
    for examples, news, stock market, weather updates etc.

    Args:
    query: The search query
    """

    search = TavilySearchResults(
        max_results=5,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=True,
        include_images=True,
    )
    response = search.invoke(query)
    return response

In [41]:
### General Final Result with Tool calling
from langchain_core.messages import HumanMessage

# 파이썬 함수들을 리스트로 구성하고 LLM에 바인딩한다
tools = [wikipedia_search, pubmed_search, tavily_search, add, multiply]

list_of_tools = { tool.name: tool for tool in tools }           # 이름도 저장해놓는다
llm_with_tools = llm.bind_tools(tools)

# query = "Tell me the latest news."
# query = "What is today's stock market news?"
# query = "What is LLM?"
# query = "How to treat lung cancer?"
query = "what is 2 * 3?"

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)                # 도구로 검색한다
messages.append(ai_msg)


# 도구들에서 받아온 결과를 메시지로 구성한다
for tool_call in ai_msg.tool_calls:
    name = tool_call['name'].lower()
    selected_tool = list_of_tools[name]

    # 해당 도구에 메시지를 전달하고 결과를 리스트로 보관
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)


# 도구에서 검색된 내용을 llm에게 다시 넣는다
response = llm_with_tools.invoke(messages)
print(response.content)

The result of the calculation 2 * 3 is 6, and I've formatted the answer as a response to your original question. Let me know if you have any further questions or if there's anything else I can help with!
